# Исследование надежности заемщиков

**Цель исследования** — определение влияния параметров клиента на факт погашения кредита в срок.

**Задачи исследования** – определить наличие/отсутствие зависимости между:
   1. *Количеством детей* и возвратом кредита в срок;
   2. *Семейным положением* и возвратом кредита в срок;
   3. *Уровнем дохода* и возвратом кредита в срок;
   4. *Целями кредита* и возвратом кредита в срок.
    
**Ход исследования**

 1. Обзор данных.
 2. Предобработка данных.
    - Заполнение пропусков
    - Проверка данных на аномалии и исправления
    - Изменение типов данных
    - Удаление дубликатов
    - Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма
    - Категоризация доходов
    - Категоризация целей кредита
 3. Проверка гипотез (Ответы на вопросы).
 4. Выводы.

## Обзор данных

In [5]:
# Импорт библиотеки pandas
import pandas as pd

In [6]:
# Чтение csv и его обзор
data = pd.read_csv('data.csv')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [7]:
# Вывод обобщенной информации о датафрейме
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


- Общее количество строк 21 525;
- Общее количество колонок 12;
- В столбцах `'days_employed'` и `'total_income'` ненулевых значений меньше, чем в остальных столбцах;
- В столбце `days_employed` набдюдаются отрицательные значения и шестизначные числа.

## Предобработка данных

### Заполнение пропусков

In [8]:
# Поиск столбцов с пропущенными значаениями и их сумма по столбцам
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

- Обнаружено два столбца с пропущенными значениями `days_employed` и `total_income` в количестве 2174. 
- Сумма пропущенных значений в столбцах совпадает и составляет 2174 значения.

In [9]:
# Расчет доли пропущенных значений в столбце days_employed
round((data['days_employed'].isna().sum() / data['days_employed'].count()), 2)

0.11

In [10]:
# Расчет доли пропущенных значений в столбце total_income
round((data['total_income'].isna().sum() / data['total_income'].count()), 2)

0.11

Пропуски занчений могут быть связаны с:

    1. Отсутствием опыта работы и доходов
    2. Отсутствием информации в базе данных
    
Считаю, что первое мловероятно, т.к. без подтверждения заработной платы клиенту не дали бы кредит.

Доля пропущенных значений в рассматриваемых столбцах составляет 0.11. Доля достаточно велика, поэтому их пропуски следует заполнить медианным значением. При удалении же строк мы потерям дополнительную информацию из других столбцов. Однако, расчет медеианного значения будет не верным для столбца `days_employed` из-за наличия отрицательных значений в нём.

**Медиана** — это такое число в выборке, что ровно половина элементов
больше него, а другая половина — меньше.

In [11]:
# Замена отрицательных значений на положительные в столбце days_employed
data['days_employed'] = abs(data['days_employed'])

In [12]:
# Проверка
len(data[data['days_employed'] < 0])

0

In [13]:
#Заполнение пропусков медианным значением в столбце days_employed
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())

In [14]:
#Заполнение пропусков медианным значением в столбце total_income
data['total_income']= data['total_income'].fillna(data['total_income'].median())

In [15]:
data['days_employed'].median()

2194.220566878695

In [16]:
data['total_income'].median()

145017.93753253992

In [17]:
#Проверка количтва пропущенных значений
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Теперь пропущенные значения отсутствуют.

**Медиана**
- `days_employed` =  2194
- `total_income` = 14518

### Проверка данных на аномалии и исправления

#### Содержание

- обзор аномалий
- children
- dob_years
- days_employed
  - отрицательные значения
  - минимальные и максимальные аномалии

#### обзор аномалий

In [18]:
# Составляем лист с интересующими столбцами
category_columns = [
                    'children', 
                    'dob_years', 
                    'education', 
                    'education_id',
                    'family_status',
                    'family_status_id',   
                    'gender', 
                    'income_type', 
                    'purpose'
                   ]
# Составляем списки уникальных значений
for column in category_columns:
    try:
        print(column, sorted(data[column].unique()))
    except:
        print('В датафрейме отсутствуюет значение колоники', column)

children [-1, 0, 1, 2, 3, 4, 5, 20]
dob_years [0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]
education ['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее', 'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']
education_id [0, 1, 2, 3, 4]
family_status ['Не женат / не замужем', 'в разводе', 'вдовец / вдова', 'гражданский брак', 'женат / замужем']
family_status_id [0, 1, 2, 3, 4]
gender ['F', 'M', 'XNA']
income_type ['безработный', 'в декрете', 'госслужащий', 'компаньон', 'пенсионер', 'предприниматель', 'сотрудник', 'студент']
purpose ['автомобили', 'автомобиль', 'высшее образование', 'дополнительное образование', 'жилье', 'заняться высшим образованием', 'заняться образованием', 'на покуп

- `children` – выбиваются два значения -1 и 20. -1 и 20 скорее всго опечатки. Проверим строку/и с этим значением.
- `dob_years` – наблюдается нулевое значение.
- `ducation` – разный регистр
- `gender` – вопрос с 'XNA' - возможно пол решили скрыть.

In [19]:
for col in data.select_dtypes(include='object').columns:
    print(col, data[col].unique(), '\n')

education ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень'] 

family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем'] 

gender ['F' 'M' 'XNA'] 

income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете'] 

purpose ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'опе

#### children

In [20]:
# Подсчет количества значений 'children' = -1
len(data.loc[data['children'] == -1])

47

In [21]:
# Обзор строк со значением 'children' = -1
data.loc[data['children'] == -1].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,2194.220567,57,Среднее,1,женат / замужем,0,F,пенсионер,0,145017.937533,на покупку своего автомобиля


Строк со значением `-1` 47 штук. Будем исходить из гипотезы того, что это опечатка в виде добавленного минуса. 

In [22]:
# Подсчет количества значений 'children' = 20
len(data.loc[data['children'] == 20])

76

In [23]:
# Обзор строк со значением 'children' = 20
data.loc[data['children'] == 20].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля


Строк со значением `20` 76 штук. 
Будем исходить из гипотезы того, что это опечатка в виде добавленного нуля. 

**Возможно рассмотрение двух вариантов:**
  1. Замена 20 на 2 and/or -1 на 1
  2. Удаление строк `children` = 20 and/or `children` = -1

В данном случае оба этих варианта не сильно повлияют на статистические характеристики выборки, т.к. анаомальных значений немного. Выберем замену.

In [24]:
# Замена значений -1 на 1 и 20 на 2 в столбце children
data['children'] = abs(data['children'])
data.loc[data['children'] == 20, 'children'] = 2

In [25]:
# Проверка
sorted(data['children'].unique())

[0, 1, 2, 3, 4, 5]

Значения -1 и 20 теперь отсутствуют.

#### dob_years

Как указывалось выше в столбце `dob_years` наблюдаются значения равные нулю.

In [26]:
# Рассчет доли значений days_employed 'dob_years' = 0
round(len(data.loc[data['dob_years'] == 0]) / data['dob_years'].count(), 3)

0.005

Доля рассматриваемого значения невелика. Соответственно их можно удалить.

In [27]:
# Сохранение датафрейма без dob_years' = 0
data = data[data['dob_years'] != 0]

In [28]:
# Проверка
len(data.loc[data['dob_years'] == 0])

0

Значение `0` теперь отсутствует.

#### days_employed

В столбце `days_employed` обнаружены:
1. Отрицательные значения
2. Аномальные максимальные и минимальные значения

##### Отрицательные значения

Отрицательные значения могут быть связаны с опечаткой и уже были заменены на этапе заполнения пропусков.

#### Минимальные и максимальные аномалии

**Минимальные аномалии**

In [29]:
# Сортировка столбца days_employed
data.sort_values(by='days_employed', ascending=True).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17437,1,24.141633,31,среднее,1,женат / замужем,0,F,сотрудник,1,166952.415427,высшее образование
8336,0,24.240695,32,высшее,0,Не женат / не замужем,4,M,сотрудник,0,124115.373655,получение дополнительного образования
6157,2,30.195337,47,среднее,1,гражданский брак,1,M,компаньон,0,231461.185606,свадьба
9683,0,33.520665,43,среднее,1,Не женат / не замужем,4,M,сотрудник,1,128555.897209,приобретение автомобиля
2127,1,34.701045,31,высшее,0,женат / замужем,0,F,компаньон,0,90557.994311,получение образования


In [30]:
# Оценка количества аномальных минимальных значений
round((data[data['days_employed'] < 365]['days_employed'].count() / len(data)), 3)

0.085

Клиентов со стажем работы менее одного года составляет чуть больше 8% от общего клиентов. Так как этих значений достаточно много, удалять их не рекомендуется.

Однозначную методику отсеивания минимальных аномалий в `days_employed` придумать затруднительно, т.к. нужно явно понимать причины таких значений. Не зная такой причины следует принять гипотезу о том, что эти данные правильные. Клиент мог работать большую часть трудовой жизни неофициально. 

**Максимальные значения**

In [31]:
# Фильтрация аномальных максимальных значений
# Выбираются (фильтруются) те значения days_employed, которые превышают возможный рабочий стаж в днях (dob_years - 14)
# 14 лет это минимальный возраст для принятия на работу, с соблюдением определенных условий.

data_employed_filter = data[data['days_employed'].astype('int') / 365 >= (data['dob_years'] - 14)]
data_employed_filter.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


На глаза бросается длинна значений отфильтрованного столбца 'days_employed', равная 6.

In [32]:
# Cреднее значение дней стажа 
data_employed_filter['days_employed'].mean() / 365

993.8315550711751

In [33]:
# Проверка длины значений
pd.options.mode.chained_assignment = None
data_employed_filter['len_days_employed'] = data_employed_filter['days_employed'].apply(lambda x: len(str(int(x))))
data_employed_filter.groupby(by='len_days_employed')['len_days_employed'].count()

len_days_employed
4      11
5      11
6    3428
Name: len_days_employed, dtype: int64

Длинна значений в отфильтрованном датафрейме перимущественно равна шести.

Проверим гипотезу о том, что запята в данных стоит не в том месте. Разделим столбец `days_employed` на 10 и на 100, и найдем разницу между `days_employed` и (`dob_years` - 14). Она должна стремиться к 0. Значения приводятся к годам.

In [34]:
# Деление days_employed на 10
# Сопоставление со значением столбца dob_years
pd.options.mode.chained_assignment = None
data_employed_filter['difference'] = (data_employed_filter['days_employed'] / 10 /365) - (data_employed_filter['dob_years'] - 14)
data_employed_filter['difference'].mean()

54.09735840566841

In [35]:
# Деление days_employed на 100
# Сопоставление со значением столбца dob_years
pd.options.mode.chained_assignment = None
data_employed_filter['difference'] = (data_employed_filter['days_employed'] / 100 /365) - (data_employed_filter['dob_years'] - 14)
data_employed_filter['difference'].mean()

-35.34748155073749

Значение разницы `days_employed` и (`dob_years` - 14) получилось сильно отличным от нуля. Гипотезу отвергаем.

Гипотезу неверном порядке значений отвергаем.

Попробуем посмотреть каким категориям присущи максимальные аномальные значения.

In [36]:
# Сводная таблица по family_status и income_type
data_employed_filter.pivot_table(index='income_type', columns = 'family_status', values='debt', aggfunc= 'count')

family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем
income_type,,,,,
безработный,NaN,NaN,NaN,1.0,1.0
госслужащий,NaN,NaN,NaN,NaN,1.0
компаньон,1.0,NaN,NaN,NaN,5.0
пенсионер,309.0,197.0,480.0,576.0,1864.0
сотрудник,1.0,NaN,NaN,4.0,10.0


Как видно отфильтрованным данным присущ тип занаятости - `пенсионер`.

Удалим из исходного датафрейма отсортированные строки с `income_type` != `пенсионер`.

In [37]:
# Удаление из исходного датафрейма аномальных максимальных значений 'income_type' != 'пенсионер'
data = data.drop(data[((data['days_employed'].astype('int') / 365) >= (data['dob_years'] - 14))  & (data['income_type'] != 'пенсионер')].index)

Рассчитаем медианное значение для пенсионеров по исходным данным без анаомальных значений.

In [38]:
# Фильтрация данных для подсчета медианного значения 
#'income_type' == 'пенсионер' по исходным данным без анаомальных значений
data_pensioner = data[(data['days_employed'].astype('int') / 365 <= (data['dob_years'] - 14)) &
                     (data['income_type'] == 'пенсионер')]

In [39]:
# Вывод медианного значения
data_pensioner['days_employed'].median()

2194.220566878695

Медиана совпала 'исходной' медианой, рассчитанной в 0.2  Шаг 2.1 Заполнение пропусков.

Заменим медианное значение.

In [40]:
# Вставка медианного значения вместо аномальных значений
data.loc[(data['days_employed'].astype('int') / 365 >= (data['dob_years'] - 14)) &
     (data['income_type'] == 'пенсионер'), 'days_employed'] = data_pensioner['days_employed'].median()

In [41]:
# Проверка
data[data['income_type'] == 'пенсионер'].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,2194.220567,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
12,0,2194.220567,65,среднее,1,гражданский брак,1,M,пенсионер,0,145017.937533,сыграть свадьбу
18,0,2194.220567,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,2194.220567,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,2194.220567,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости


### Изменение типов данных

In [42]:
#Замена вещественного типа данных на целочисленный в столбце total_income
data['total_income'] = data['total_income'].astype('int')
data['total_income'].dtype

dtype('int32')

In [43]:
#Замена вещественного типа данных на целочисленный в столбце days_employed
data['days_employed'] = data['days_employed'].astype('int')
data['days_employed'].dtype

dtype('int32')

### Удаление дубликатов

**Неявные дубликаты**

In [44]:
# Обработка неявных дубликатов в столбце education (приведение к одному регистру)
data['education'] = data['education'].str.lower()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,2194,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Явные дубликаты**

In [45]:
# Подсчет явных дубликатов
data.duplicated().sum()

71

Явные дубликаты есть и их 54 штуки.

In [46]:
# Удаление явных дубликатов
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

Функция duplicated() возвращает булевые значения для строки, значения в которой уже были выше.
Функция duplicated() с методом sum() подсчитывает количество дубликатов.
Функция drop_duplicates() удаляет явные дубликаты, а метод reset_index(drop=True) создает новый столбец с индексами. Аргумент drop=True указывают, чтобы не создавать столбец
со старыми значениями индексов.

Появление явных дубликатов может быть связано с повторным обращением клиента или могло произойти при слиянии баз данных.

Появление неявных дубликатов
Отсутствие четкого регламента заполнения таблицы

**Неявные дубликаты**

Рассмотрим столбец `purpose`.

In [47]:
# Список уникальных значений столбца purpose
sorted((data['purpose'].unique()).astype('str'))

['автомобили',
 'автомобиль',
 'высшее образование',
 'дополнительное образование',
 'жилье',
 'заняться высшим образованием',
 'заняться образованием',
 'на покупку автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'на проведение свадьбы',
 'недвижимость',
 'образование',
 'операции с жильем',
 'операции с коммерческой недвижимостью',
 'операции с недвижимостью',
 'операции со своей недвижимостью',
 'покупка жилой недвижимости',
 'покупка жилья',
 'покупка жилья для сдачи',
 'покупка жилья для семьи',
 'покупка коммерческой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'получение высшего образования',
 'получение дополнительного образования',
 'получение образования',
 'приобретение автомобиля',
 'профильное образование',
 'ремонт жилью',
 'свадьба',
 'свой автомобиль',
 'сделка с автомобилем',
 'сделка с подержанным автомобилем',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недв

Неявные дубликаты в столбце `purpose` уйдут ниже при категоризации.

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

In [48]:
# Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма
education = data.groupby('education')['education_id'].mean()
family_status = data.groupby(by='family_status')['family_status_id'].mean()
data = data.drop(columns=['education', 'family_status'])
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,2194,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Категоризация дохода

In [49]:
# Функция для категоризации дохода
def income_category(income):
    if 0 <= income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'    
    elif 200001 <= income <= 1000000:
        return 'B'  
    else: return 'A'

In [50]:
# Применение функции income_category
data['total_income_category'] = data['total_income'].apply(income_category)
data['total_income_category'].value_counts()

C    15924
B     5010
D      348
A       25
E       22
Name: total_income_category, dtype: int64

### Категоризация целей кредита

In [51]:
# Снова рассмотрим уникальные значения purpose
sorted(data['purpose'].unique())

['автомобили',
 'автомобиль',
 'высшее образование',
 'дополнительное образование',
 'жилье',
 'заняться высшим образованием',
 'заняться образованием',
 'на покупку автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'на проведение свадьбы',
 'недвижимость',
 'образование',
 'операции с жильем',
 'операции с коммерческой недвижимостью',
 'операции с недвижимостью',
 'операции со своей недвижимостью',
 'покупка жилой недвижимости',
 'покупка жилья',
 'покупка жилья для сдачи',
 'покупка жилья для семьи',
 'покупка коммерческой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'получение высшего образования',
 'получение дополнительного образования',
 'получение образования',
 'приобретение автомобиля',
 'профильное образование',
 'ремонт жилью',
 'свадьба',
 'свой автомобиль',
 'сделка с автомобилем',
 'сделка с подержанным автомобилем',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недв

In [52]:
# Функция категоризации purpose
def purpose_category(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif 'свад' in purpose: 
        return 'проведение свадьбы'
    elif 'образован' in purpose:
        return 'получение образования'
    elif 'недвижимост' or 'жиль' in purpose:
        return 'операции с недвижимостью'

In [53]:
# Применение функции purpose_category
data['purpose_category'] = data['purpose'].apply(purpose_category)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,2194,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [54]:
# Проверка
sorted(data['purpose_category'].unique())

['операции с автомобилем',
 'операции с недвижимостью',
 'получение образования',
 'проведение свадьбы']

## Проверка гипотез

### Количество детей

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [55]:
# Сводная таблица доля клиентов с задолженностью в зависимости от количества детей в %
round(data.pivot_table(columns='children', values='debt', aggfunc='mean'), 4) * 100

children,0,1,2,3,4,5
debt,7.56,9.12,9.56,8.23,9.76,0.0


In [56]:
df_example = data.pivot_table(index = 'children', values = 'debt', 
                            aggfunc = ['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
children,,,,
0,14003,1058,7.56%,92.44%
1,4834,441,9.12%,90.88%
2,2114,202,9.56%,90.44%
3,328,27,8.23%,91.77%
4,41,4,9.76%,90.24%
5,9,0,0.00%,100.00%


**Вывод**

В целом значимой зависимости не прослеживается. Можно отметить, что при количестве детей = 0 доля клиентов с задолженностями немного меньше. Следует отметить, что клиенты с пятью детьми задолжностей не имели.

### Семейное положение

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [57]:
# Сводная таблица доля клиентов с задолженностью в зависимости от семейного положения в %
round(data.pivot_table(columns='family_status_id', values='debt', aggfunc='mean'), 4) * 100

family_status_id,0,1,2,3,4
debt,7.55,9.36,6.5,7.17,9.78


In [58]:
family_status

family_status
Не женат / не замужем    4.0
в разводе                3.0
вдовец / вдова           2.0
гражданский брак         1.0
женат / замужем          0.0
Name: family_status_id, dtype: float64

**Вывод**

Можно выделить две категории:
- С долей задолженности >= 8% (гражданский брак, Не женат / не замужем )
- С долей задолженности < 8% (женат / замужем, вдовец / вдова, в разводе)

### Уровень дохода

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [59]:
# Сводная таблица доля клиентов с задолженностью в зависимости от уровня дохода в %
round(data.pivot_table(columns='total_income_category', values='debt', aggfunc='mean'), 4) * 100

total_income_category,A,B,C,D,E
debt,8.0,7.09,8.49,6.03,9.09


Расшифровка `total_income_category`	

0–30000 — 'E';

30001–50000 — 'D';

50001–200000 — 'C';

200001–1000000 — 'B';

1000001 и выше — 'A'.

**Вывод**

Можно выделить две категории:
- С долей задолженности >= 8% (`A`, `C`, `E`)
- С долей задолженности < 8% (`B`, `D`)

### Цели кредита

Как разные цели кредита влияют на его возврат в срок?

In [60]:
# Сводная таблица доля клиентов с задолженностью в зависимости от цели кредита в %
round(data.pivot_table(columns= 'purpose_category', values='debt', aggfunc='mean'), 4) * 100

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
debt,9.34,7.24,9.27,7.97


**Вывод**

Можно выделить две категории:
- С долей задолженности >= 8% (`операции с автомобилем`, `получение образования`)
- С долей задолженности < 8% (`операции с недвижимостью`, `проведение свадьбы`)

## Общий вывод:

Были проверены четыре вопроса:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
   При количестве детей = 0 доля клиентов с задолженностями меньше. Клиенты с пятью детьми задолжностей не имеют.
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
   - Доля клиентов с задолженностями >= 8% (гражданский брак, Не женат / не замужем )
   - Доля клиентов с задолженностями < 8% (женат / замужем, вдовец / вдова, в разводе)
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
   - Доля клиентов с задолженностями >= 8% (`A`, `C`, `E`)
   - Доля клиентов с задолженностями < 8% (`B`, `D`)
4. Как разные цели кредита влияют на его возврат в срок?
   - Доля клиентов с задолженностями >= 8% (`операции с автомобилем`, `получение образования`)
   - Доля клиентов с задолженностями < 8% (`операции с недвижимостью`, `проведение свадьбы`)